In [1]:
import requests
import json
import numpy as np
import pandas as pd
from pandas import json_normalize
import geopandas as gpd

## Read in geometry data, plants data and merge them

In [2]:
with open('/Users/minjukim/Documents/GitHub/korec_energy_permitting/12-10-22-7pm.json') as json_data:
    geometry_data = json.load(json_data)

latlongs = pd.DataFrame(geometry_data)

In [3]:
df = pd.read_csv('/Users/minjukim/Documents/GitHub/korec_energy_permitting/cleaned_df_updated_address.csv')

In [4]:
df = df.sort_values(by=['address']).reset_index(drop=True)

In [24]:
df.sort_values(by=['capacity_MW'], ascending=False)

,plant_count,geometry.coordinates,capacity_MW,cleaned_type,properties.article
364,1,"[126.5191246, 36.4007416]",5800.0,coal,<div class='plant_list'><P>Type: coal\nCapacit...
94,2,"[126.5096353, 37.0326652]",5000.0,coal,<div class='plant_list'><P>Type: coal\nCapacit...
379,1,"[126.2247896, 36.8948887]",4000.0,coal,<div class='plant_list'><P>Type: coal\nCapacit...
71,2,"[126.5908916, 36.9819022]",3150.0,coal,<div class='plant_list'><P>Type: coal\nCapacit...
742,1,"[128.4245023, 36.7573156]",2800.0,nuclear,<div class='plant_list'><P>Type: nuclear\nCapa...
...,...,...,...,...,...
555,1,"[127.0095212, 37.8931512]",3.2,coal,<div class='plant_list'><P>Type: coal\nCapacit...
864,1,"[128.7584813, 36.1020786]",3.1,others,<div class='plant_list'><P>Type: others\nCapac...
321,1,"[126.8399033, 36.3047454]",3.1,solar,<div class='plant_list'><P>Type: solar\nCapaci...
366,1,"[126.9506547, 36.2133201]",3.1,solar,<div class='plant_list'><P>Type: solar\nCapaci...


In [6]:
latlongs = latlongs.sort_values(by=['place']).reset_index(drop=True)

,place,geometry.type,geometry.coordinates
0,"강원 강릉시 강동면 임곡리 및 언별리, 옥계면 북동리, 왕산면 목계...",Point,"[128.932449, 37.691505]"
1,"강원 강릉시 강동면 임곡리 및 언별리, 옥계면 북동리, 왕산면 목계리...",Point,"[128.932449, 37.691505]"
2,강원 강릉시 강동면 임곡리 산21 일원,Point,"[128.9920227, 37.6883164]"
3,"강원 강릉시 옥계면 금진리 산81번지, 강동면 산성우리 산56번지",Point,"[129.0318912, 37.6441514]"
4,강원 강릉시 옥계면 산계리 8번지(라파즈한라시멘트 공장부지 내),Point,"[129.008498, 37.589326]"
...,...,...,...
1084,충북 진천군 광혜원면 광혜원리 773-20번지 일원,Point,"[127.4355576, 36.9800267]"
1085,충북 청원군 옥산면국사리,Point,"[127.3744217, 36.687463]"
1086,충청남도 부여군 장암면석동리 산42-1,Point,"[126.8920177, 36.240187]"
1087,"충청북도 영동군 상촌면 고자리 일원, 양강면 산막리 일원",Point,"[127.8396796, 36.1002673]"


In [7]:
df = df.join(latlongs).drop(['place'], axis=1)

In [9]:
df.to_csv('df_with_latlong.csv', index=False)

## Prep joined dataframe into correct format

In [10]:
df = pd.read_csv("df_with_latlong.csv")

## Treat overlapping coordinates by manually adjusting them into more accurate coords

In [11]:
coord_has_multiple_plants = df["geometry.coordinates"].value_counts() > 1

coord_has_multiple_plants.keys()

overlapping_coords = df[df['geometry.coordinates'].isin(coord_has_multiple_plants.keys())]

overlapping_coords.groupby('geometry.coordinates')['cleaned_type'].nunique().sort_values(ascending=False).head(60)

geometry.coordinates
[129.3113596, 35.5383773]                   4
[128.8315766, 36.5156317]                   3
[127.5376194, 34.9138101]                   2
[127.1634514, 37.5683014]                   2
[128.3750314, 35.2714244]                   2
[128.3724286, 36.0970442]                   2
[126.482758, 35.820222]                     2
[129.0509505, 37.0942103]                   2
[126.4335439, 37.051032]                    2
[126.4014369, 34.8325814]                   2
[127.153, 35.71750000000001]                2
[127.1618493, 37.6880177]                   2
[126.362671, 36.9959976]                    2
[126.5341796, 35.9480587]                   2
[126.3558487, 35.2625367]                   2
[127.1956234, 34.5390034]                   2
[127.8520885, 34.9764065]                   2
[127.2574272, 34.6431854]                   2
[126.301154, 34.9066445]                    2
[127.3183346, 34.0485456]                   2
[127.341032, 35.2313017]                    2
[126.5317163,

In [12]:
'''
[129.3113596, 35.5383773]                   4
Types: others, wind, gas, coal
'''
gas = (df['geometry.coordinates']=='[129.3113596, 35.5383773]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[129.35849908473662, 35.51559397107106]'

wind = (df['geometry.coordinates']=='[129.3113596, 35.5383773]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[129.51648868036636, 35.53865916003904]'

others = (df['geometry.coordinates']=='[129.3113596, 35.5383773]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[129.3583561559003, 35.51942439324197]'
'''

[128.8315766, 36.5156317]                   3
Types: solar, others, gas
'''
solar = (df['geometry.coordinates']=='[128.8315766, 36.5156317]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[128.83163024243228, 36.51575239781046]'

others = (df['geometry.coordinates']=='[128.8315766, 36.5156317]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[128.48787354026564, 36.5598796059351]'
'''

[127.5376194, 34.9138101]                   2
Types: others, gas
'''
others = (df['geometry.coordinates']=='[127.5376194, 34.9138101]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[127.56729314632275, 34.90236694573741]'

'''
[127.1634514, 37.5683014]                   2
Types: others, gas
'''
others = (df['geometry.coordinates']=='[127.1634514, 37.5683014]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[127.16612588478432, 37.568541650846356]'
'''

[128.3750314, 35.2714244]                   2
Types: others, gas
'''
others = (df['geometry.coordinates']=='[128.3750314, 35.2714244]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[128.46691287473, 35.34389724940427]'

'''
[128.3724286, 36.0970442]                   2
Types: others, wind
'''
others = (df['geometry.coordinates']=='[128.3724286, 36.0970442]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[128.44236956936126, 36.15219320273989]'

'''
[126.482758, 35.820222]                     2
Types: wind, solar
'''
solar = (df['geometry.coordinates']=='[126.482758, 35.820222]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[126.48227263966949, 35.81943934257905]'

'''
[129.0509505, 37.0942103]                   2
Types: solar, others
'''
solar = (df['geometry.coordinates']=='[129.0509505, 37.0942103]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[129.05226506190158, 37.09404046309632]'
'''

[126.4335439, 37.051032]                    2
Types: solar, wind
'''
wind = (df['geometry.coordinates']=='[126.4335439, 37.051032]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[126.45722804027132, 37.04326768372247]'
'''

[126.4014369, 34.8325814]                   2
Types: solar, others
'''
solar = (df['geometry.coordinates']=='[126.4014369, 34.8325814]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[126.75097144981373, 34.980052740815424]'

'''
[127.153, 35.71750000000001]                2
Types: others, solar
'''
solar = (df['geometry.coordinates']=='[127.153, 35.71750000000001]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[126.50682839439833, 35.823199630607526]'

'''
[127.1618493, 37.6880177]                   2
Types: gas, others
'''
others = (df['geometry.coordinates']=='[127.1618493, 37.6880177]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[127.16114853089665, 37.68716996473012]'

'''
[126.362671, 36.9959976]                    2
Types: gas, bioenergy
'''
gas = (df['geometry.coordinates']=='[126.362671, 36.9959976]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[126.36284265964306, 36.99595473549502]'
'''

[126.5341796, 35.9480587]                   2
Types: bioenergy, others
'''
others = (df['geometry.coordinates']=='[126.5341796, 35.9480587]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[126.54830487310984, 35.95977778013028]'

'''
[126.3558487, 35.2625367]                   2
Types: wind, bioenergy
'''
wind = (df['geometry.coordinates']=='[126.3558487, 35.2625367]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[126.33647374277385, 135.280962605460395]'

'''
[127.1956234, 34.5390034]                   2
Types: others, solar
'''
solar = (df['geometry.coordinates']=='[127.1956234, 34.5390034]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[126.50682839439833, 35.823199630607526]'
'''

[127.8520885, 34.9764065]                   2
Types: others, gas
'''
gas = (df['geometry.coordinates']=='[127.8520885, 34.9764065]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[127.76199014239887, 35.06101272032883]'

'''
[127.2574272, 34.6431854]                   2
Types: wind, solar
'''
solar = (df['geometry.coordinates']=='[127.2574272, 34.6431854]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[127.25641356604383, 34.63914665801225]'

'''
[126.301154, 34.9066445]                    2
Types: solar, wind
'''
solar = (df['geometry.coordinates']=='[126.301154, 34.9066445]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[126.05282459555036, 34.76208539319091]'

'''
[127.3183346, 34.0485456]                   2
Types: wind, solar
'''
solar = (df['geometry.coordinates']=='[127.3183346, 34.0485456]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[126.5049320884225, 37.01425171215782]'
'''

[127.341032, 35.2313017]                    2
Types: bioenergy, wind
'''
wind = (df['geometry.coordinates']=='[127.341032, 35.2313017]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[127.33106951356653, 35.23747719140524]'

'''
[126.5317163, 35.9596094]                   2
Types: solar, wind
'''
wind = (df['geometry.coordinates']=='[126.5317163, 35.9596094]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[126.58653548289253, 35.96195535616739]'
'''

[128.987596, 36.2210961]                    2
Types: wind, gas
'''
wind = (df['geometry.coordinates']=='[128.987596, 36.2210961]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[128.98759598455467, 36.221208536595]'

'''
[127.7328376, 36.4240364]                   2
Types: gas, others
'''
gas = (df['geometry.coordinates']=='[127.7328376, 36.4240364]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[127.73308436148355, 36.424355796735604]'
'''

[126.9661578, 34.5120061]                   2
Types: wind, others
'''
wind = (df['geometry.coordinates']=='[126.9661578, 34.5120061]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[126.90627574053796, 34.76441058373449]'
'''

[128.7584813, 36.1020786]                   2
Types: wind, others
'''
wind = (df['geometry.coordinates']=='[128.7584813, 36.1020786]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[128.75849202707747, 36.10225195113703]'
'''

[126.7439264, 34.67187020000001]            2
Types: bioenergy, wind
'''
wind = (df['geometry.coordinates']=='[126.7439264, 34.67187020000001]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[127.18224354014313, 35.04870051729036]'
'''

[126.741426, 35.114326]                     2
Types: solar, gas
'''
gas = (df['geometry.coordinates']=='[126.741426, 35.114326]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[126.76954368287392, 35.138275232866604]'

'''
[128.7244618, 35.3839605]                   2
Types: others, gas
'''
gas = (df['geometry.coordinates']=='[128.7244618, 35.3839605]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[128.72742297979983, 35.383650284311656]'

'''
[126.7221482, 36.0087855]                   2
Types: bioenergy, others
'''
others = (df['geometry.coordinates']=='[126.7221482, 36.0087855]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[126.72125276441479, 36.006741210034406]'

'''
[128.6685978, 36.3619049]                   2
Types: gas, solar
'''
solar = (df['geometry.coordinates']=='[128.6685978, 36.3619049]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[128.66822605591983, 36.3656026892897]'

'''
[126.7813551, 36.9571468]                   2
Types: bioenergy, gas
'''
gas = (df['geometry.coordinates']=='[126.7813551, 36.9571468]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[126.7813121693573, 36.95729246089392]'
'''

[128.857084, 37.0701661]                    2
Types: gas, wind
'''
wind = (df['geometry.coordinates']=='[128.857084, 37.0701661]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[128.85753777923878, 37.07111002332878]'

'''
[126.8297814, 35.19685190000001]            2
Types: others, bioenergy
'''
others = (df['geometry.coordinates']=='[126.8297814, 35.19685190000001]') & (df.cleaned_type=='others')
df.loc[others, 'geometry.coordinates'] = '[126.7811923670608, 35.027581545629864]'

'''
[126.6534466, 37.4903306]                   2
Types: gas, others
'''
gas = (df['geometry.coordinates']=='[126.6534466, 37.4903306]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[126.63189128483583, 37.61067909955937]'

'''
[126.6044483, 37.6131592]                   2
Types: gas, bioenergy
'''
gas = (df['geometry.coordinates']=='[126.6044483, 37.6131592]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[126.58919655628783, 37.60363053509413]'

'''
[126.8852807, 36.982824]                    2
Types: others, gas
'''
gas = (df['geometry.coordinates']=='[126.8852807, 36.982824]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[126.8526993949864, 36.98239781577841]'

'''
[126.8896786, 34.3793858]                   2
Types: wind, solar
'''
wind = (df['geometry.coordinates']=='[126.8896786, 34.3793858]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[127.02026875169426, 34.23904309330645]'

'''
[128.4601957, 35.7406053]                   2
Types: gas, solar
'''
gas = (df['geometry.coordinates']=='[128.4601957, 35.7406053]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[128.45283971143073, 35.68497350350508]'

'''
[126.5908916, 36.9819022]                   2
Types: solar, coal
'''
solar = (df['geometry.coordinates']=='[126.5908916, 36.9819022]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[126.55721775212004, 37.0120809188898]'

'''
[126.9369547, 35.2263093]                   2
Types: solar, others
'''
solar = (df['geometry.coordinates']=='[126.9369547, 35.2263093]') & (df.cleaned_type=='solar')
df.loc[solar, 'geometry.coordinates'] = '[126.93693324054836, 35.226458271102985]'

'''
[126.5618458, 34.5034189]                   2
Types: wind, solar
'''
wind = (df['geometry.coordinates']=='[126.5618458, 34.5034189]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[126.55245944238266, 34.33434145073902]'
'''

[129.3562888, 35.4647551]                   2
Types: others, gas
'''
gas = (df['geometry.coordinates']=='[129.3562888, 35.4647551]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[129.38300419822747, 35.51079188047425]'

'''
[128.7449683, 36.5870676]                   2
Types: gas, solar
'''
gas = (df['geometry.coordinates']=='[128.7449683, 36.5870676]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[128.7450326576457, 36.58727426949677]'

'''
[127.5877061, 34.9358814]                   2
Types: others, gas
'''
gas = (df['geometry.coordinates']=='[127.5877061, 34.9358814]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[127.58484353565251, 34.94081791545309]'

'''
[129.3666775, 35.5547591]                   2
Types: coal, gas
'''
gas = (df['geometry.coordinates']=='[129.3666775, 35.5547591]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[129.36711738051733, 35.55495983669951]'

'''
[127.7171754, 37.2551337]                   2
Types: others, gas
'''
gas = (df['geometry.coordinates']=='[127.7171754, 37.2551337]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[126.8541708304813, 37.380890233749035]'

'''
[127.5619774, 34.8996473]                   2
Types: others, gas
'''
gas = (df['geometry.coordinates']=='[127.5619774, 34.8996473]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[127.56720731563227, 34.90236694573741]'

'''
[129.4321372, 36.1106446]                   2
Types: gas, wind
'''
gas = (df['geometry.coordinates']=='[129.4321372, 36.1106446]') & (df.cleaned_type=='gas')
df.loc[gas, 'geometry.coordinates'] = '[129.4146434270773, 36.099726687870195]'

'''
[127.578106, 34.8996486]                    2
Types: wind, solar
'''
wind = (df['geometry.coordinates']=='[127.578106, 34.8996486]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[127.28743634054801, 35.20917213680167]'

'''
[126.111226, 35.08416450000001]             2
Types: solar, wind
'''
wind = (df['geometry.coordinates']=='[126.111226, 35.08416450000001]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[126.05438143373256, 35.106422217085864]'

'''
[126.04729041675004, 34.656425822471995]    2
Types: gas, wind
'''
wind = (df['geometry.coordinates']=='[126.04729041675004, 34.656425822471995]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[126.15297861555213, 34.98346444218035]'

'''
[126.046462, 34.8852314]                    2
Types: solar, wind
'''
wind = (df['geometry.coordinates']=='[126.046462, 34.8852314]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[126.09906735500746, 34.915526935807414]'

'''
[129.4057461, 36.4540751]                   2
Types: wind, gas
'''
wind = (df['geometry.coordinates']=='[129.4057461, 36.4540751]') & (df.cleaned_type=='wind')
df.loc[wind, 'geometry.coordinates'] = '[129.41702488475838, 36.465209216710385]'

## Reduce down dataframe into each coordinate

In [14]:
df["string"] = "Type: " + df['cleaned_type'] + "\nCapacity: " + df['capacity_MW'].astype(str) + " MW\nAddress: " + df['address'].str.strip() + "\nCompany: " + df['company'] + "\nOwner: " + df['owner'] + "\nApproved year: " + df['year'].astype(str)
df["string"] = df["string"].astype(str)

In [16]:
output = df.groupby('geometry.coordinates')['string'].apply(lambda x: "<div class='plant_list'><P>{0}</P></div>".format('</p><p> '.join(x))).reset_index(name='properties.article')

In [18]:
count_per_coord = df['geometry.coordinates'].value_counts().reset_index(name='plant_count')
capacity_per_coord = df.groupby('geometry.coordinates')['capacity_MW'].sum().reset_index()
type_per_coord = pd.DataFrame(df.groupby(["geometry.coordinates", "cleaned_type"], as_index=False)["cleaned_type"].first())

In [19]:
joined = pd.merge(count_per_coord, capacity_per_coord, left_on='index', right_on="geometry.coordinates").drop(['index'], axis=1)

In [20]:
joined = joined.merge(type_per_coord)

In [21]:
df = joined.merge(output)

In [23]:
df.to_csv('coordinate_df.csv', index=False)